In [222]:
import pandas as pd
import chart_studio.plotly as py
import plotly.express as px
import plotly.graph_objects as go
import numpy as np
import os
import matplotlib.pyplot as plt
from ast import literal_eval
from numpy import array
from src.utils.utils_funcs import load_init_states
from pydantic import BaseModel, validator
from pathlib import Path
from typing import List, Optional, Union, Tuple, Dict, Any
from collections import OrderedDict

In [223]:
class ParametersBounds(BaseModel):
    Kp: Tuple[float, float]
    Ki: Tuple[float, float]
    Kd: Tuple[float, float]


class Constraint(BaseModel):
    overshoot: Tuple[float, float]
    risetime: Tuple[float, float]


class Gains(BaseModel):
    Kp: float
    Ki: float
    Kd: float


class TxtContent(BaseModel):
    parameters_bounds: ParametersBounds
    constraint: Constraint
    n_iter: int
    n_trials: int
    experiment_total_run_time: int
    experiment_values_dump_rate: int
    selected_init_state: int
    objective_value_limit_early_stop: float
    total_exp_time: float
    x: Gains
    settling_time: int


class Trial(BaseModel):
    init_0_csv: Any
    init_1_csv: Any
    init_2_csv: Any
    init_3_csv: Any
    init_4_csv: Any
    init_5_csv: Any
    init_0_txt: Optional[TxtContent] = None
    init_1_txt: Optional[TxtContent] = None
    init_2_txt: Optional[TxtContent] = None
    init_3_txt: Optional[TxtContent] = None
    init_4_txt: Optional[TxtContent] = None
    init_5_txt: Optional[TxtContent] = None

    @validator(
        "init_0_csv",
        "init_1_csv",
        "init_2_csv",
        "init_3_csv",
        "init_4_csv",
        "init_5_csv",
        pre=True,
        always=True,
    )
    def validate_csv(cls, v):
        if not isinstance(v, pd.DataFrame):
            raise ValueError("CSV file must be a pandas DataFrame")
        return v


class Config(BaseModel):
    trial_1: Trial
    # trial_2: Optional[Trial] = None
    # trial_3: Optional[Trial] = None
    configs_file: Path


class OptimizerResults(BaseModel):
    config_1: Config
    config_2: Config
    config_3: Config


class DifferentialEvolutionResults(OptimizerResults):
    pass


class BayesianOptimizerResults(OptimizerResults):
    pass

/var/folders/f6/c9bxgxqd72ndbxyxv7_8mcrh0000gp/T/ipykernel_26155/744051963.py:46: PydanticDeprecatedSince20: Pydantic V1 style `@validator` validators are deprecated. You should migrate to Pydantic V2 style `@field_validator` validators, see the migration guide for more details. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.5/migration/
  @validator(


In [224]:
BASE_PATH = "/Users/aalbustami/Ali/PID-Tuning-for-Motion-Optimization"

de_dir = os.path.join(BASE_PATH, "DE-results")
bo_dir = os.path.join(BASE_PATH, "BO-results")

de_results = {}

In [225]:
# configs = ["config_1", "config_2", "config_3"]
# trials = {"config_1": "trial_1", "config_2": "trial_1", "config_3": "trial_1"}
# initializations = [f"init_{i}" for i in range(6)]
# file_types = ["csv", "txt"]

# # Function to generate the dictionary structure
# def create_dict(configs, trials, initializations, file_types):
#     return {
#         config: {
#             trial: {f"{init}_{file_type}": None for init in initializations for file_type in file_types}
#             for trial in [trials[config]]
#         }
#         for config in configs
#     }

# # Create de_dict and do_dict using the function
# de_dict = create_dict(configs, trials, initializations, file_types)
# do_dict = create_dict(configs, trials, initializations, file_types)

# print(de_dict)
# print(do_dict)

In [226]:
# de_dict = {
#     "config_1": {
#         "trial_1": {
#             "init_0_csv": None,
#             "init_1_csv": None,
#             "init_2_csv": None,
#             "init_3_csv": None,
#             "init_4_csv": None,
#             "init_5_csv": None,
#             "init_0_txt": None,
#             "init_1_txt": None,
#             "init_2_txt": None,
#             "init_3_txt": None,
#             "init_4_txt": None,
#             "init_5_txt": None,
#         },
#         "configs_file": None,
#     },
#     "config_2": {
#         "trial_1": {
#             "init_0_csv": None,
#             "init_1_csv": None,
#             "init_2_csv": None,
#             "init_3_csv": None,
#             "init_4_csv": None,
#             "init_5_csv": None,
#             "init_0_txt": None,
#             "init_1_txt": None,
#             "init_2_txt": None,
#             "init_3_txt": None,
#             "init_4_txt": None,
#             "init_5_txt": None,
#         },
#         "configs_file": None,
#     },
#     "config_3": {
#         "trial_1": {
#             "init_0_csv": None,
#             "init_1_csv": None,
#             "init_2_csv": None,
#             "init_3_csv": None,
#             "init_4_csv": None,
#             "init_5_csv": None,
#             "init_0_txt": None,
#             "init_1_txt": None,
#             "init_2_txt": None,
#             "init_3_txt": None,
#             "init_4_txt": None,
#             "init_5_txt": None,
#         },
#         "configs_file": None,
#     },
# }

# do_dict = {
#     "config_1": {
#         "trial_1": {
#             "init_0_csv": None,
#             "init_1_csv": None,
#             "init_2_csv": None,
#             "init_3_csv": None,
#             "init_4_csv": None,
#             "init_5_csv": None,
#             "init_0_txt": None,
#             "init_1_txt": None,
#             "init_2_txt": None,
#             "init_3_txt": None,
#             "init_4_txt": None,
#             "init_5_txt": None,
#         },
#         "configs_file": None,
#     },
#     "config_2": {
#         "trial_1": {
#             "init_0_csv": None,
#             "init_1_csv": None,
#             "init_2_csv": None,
#             "init_3_csv": None,
#             "init_4_csv": None,
#             "init_5_csv": None,
#             "init_0_txt": None,
#             "init_1_txt": None,
#             "init_2_txt": None,
#             "init_3_txt": None,
#             "init_4_txt": None,
#             "init_5_txt": None,
#         },
#         "configs_file": None,
#     },
#     "config_3": {
#         "trial_1": {
#             "init_0_csv": None,
#             "init_1_csv": None,
#             "init_2_csv": None,
#             "init_3_csv": None,
#             "init_4_csv": None,
#             "init_5_csv": None,
#             "init_0_txt": None,
#             "init_1_txt": None,
#             "init_2_txt": None,
#             "init_3_txt": None,
#             "init_4_txt": None,
#             "init_5_txt": None,
#         },
#         "configs_file": None,
#     },
# }
def generate_dict(configs, trials, init_files):
    result_dict = {}

    for config in configs:
        result_dict[config] = {}
        for trial in trials:
            init_dict = {}
            for init_file in init_files:
                for i in range(6):
                    key = f"init_{i}_{init_file}"
                    init_dict[key] = None
            result_dict[config][trial] = init_dict
        result_dict[config]["configs_file"] = None

    return result_dict

# Example usage
configs = ["config_1", "config_2", "config_3"]
trials = ["trial_1"]
init_files = ["csv", "txt"]

de_dict = generate_dict(configs, trials, init_files)
bo_dict = generate_dict(configs, trials, init_files)

In [227]:
def read_txt_file(file_path):
    with open(file_path, "r") as file:
        data = file.read()
        data = eval(data)
        
        if isinstance(data["x"], np.ndarray):
            data["x"] = Gains(Kp=data["x"][0], Ki=data["x"][1], Kd=data["x"][2])

        
    return data


In [228]:
ALLOWED_CONFIGS = [f"config_{i}" for i in range(1, 4)]
INIT_STATES = [f"init_{i}" for i in range(6)]
FILE_TYPES = ["csv", "txt"]

for root, dirs, files in os.walk(de_dir):
    for file_ in files:
        if not any(
            allowed_config in root for allowed_config in ALLOWED_CONFIGS
        ):
            continue  # to skip config_4 and above

        if "configs" in file_:
            for config in ALLOWED_CONFIGS:
                if config in root:
                    de_dict[config]["configs_file"] = os.path.join(root, file_)

            continue
        config = root.split(os.sep)[-2]
        trial = root.split(os.sep)[-1]
        selected_file = (
            "_".join(file_.split("-")[-1].split("_")[1:][:2])
            + "_"
            + file_.split("-")[-1].split("_")[-1].split(".")[1]
        )
        # de_dict[config][trial][selected_file] = os.path.join(root, file_)
        if "csv" in file_:
            de_dict[config][trial][selected_file] = pd.read_csv(os.path.join(root, file_))
        elif "txt" in file_:
            results = read_txt_file(os.path.join(root, file_))
            de_dict[config][trial][selected_file] = results
        
        
for root, dirs, files in os.walk(bo_dir):
    for file_ in files:
        if not any(
            allowed_config in root for allowed_config in ALLOWED_CONFIGS
        ):
            continue  # to skip config_4 and above

        if "configs" in file_:
            for config in ALLOWED_CONFIGS:
                if config in root:
                    do_dict[config]["configs_file"] = os.path.join(root, file_)

            continue
        config = root.split(os.sep)[-2]
        trial = root.split(os.sep)[-1]
        selected_file = (
            "_".join(file_.split("-")[-1].split("_")[1:][:2])
            + "_"
            + file_.split("-")[-1].split("_")[-1].split(".")[1]
        )

        # do_dict[config][trial][selected_file] = os.path.join(root, file_)
        if "csv" in file_:
            do_dict[config][trial][selected_file] = pd.read_csv(os.path.join(root, file_))
        elif "txt" in file_:
            results = read_txt_file(os.path.join(root, file_))
            do_dict[config][trial][selected_file] = results


In [229]:
de_results = DifferentialEvolutionResults(**de_dict)
bo_results = BayesianOptimizerResults(**do_dict)

In [230]:
print(de_results.config_1.trial_1.init_1_csv)
print(bo_results.config_1.trial_1.init_1_txt.x.Kp)

   experiment_id     kp    ki      kd  overshoot  rise_time  settling_time  \
0              1  12.00  0.00  0.7800  19.877778        600           2200   
1              2  19.50  0.43  0.0010  29.855556        500           5000   
2              3  18.25  0.44  0.7525  28.955556        500           5000   

                                        angle_values  set_point  
0  [-0.11, 3.24, 9.89, 19.81, 34.05, 48.25, 66.09...         90  
1  [0.58, 9.42, 23.11, 33.12, 47.68, 66.1, 84.97,...         90  
2  [0.01, 6.16, 19.22, 31.09, 45.25, 62.62, 81.1,...         90  
18.249985695121897


In [ ]:
""